# Musicians - Hard

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app16-3")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@6fe5632b

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@57b5383f

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val band = hiveCxt.table("sqlzoo.band")
val composer = hiveCxt.table("sqlzoo.composer")
val composition = hiveCxt.table("sqlzoo.composition")
val concert = hiveCxt.table("sqlzoo.concert")
val has_composed = hiveCxt.table("sqlzoo.has_composed")
val musician = hiveCxt.table("sqlzoo.musician")
val performance = hiveCxt.table("sqlzoo.performance")
val performer = hiveCxt.table("sqlzoo.performer")
val place = hiveCxt.table("sqlzoo.place")
val plays_in = hiveCxt.table("sqlzoo.plays_in")

band: DataFrame = [band_no: int, band_name: string ... 4 more fields]
composer: DataFrame = [comp_no: int, comp_is: int ... 1 more field]
composition: DataFrame = [c_no: int, comp_date: string ... 2 more fields]
concert: DataFrame = [concert_no: int, concert_venue: string ... 3 more fields]
has_composed: DataFrame = [cmpr_no: int, cmpn_no: int]
musician: DataFrame = [m_no: int, m_name: string ... 4 more fields]
performance: DataFrame = [pfrmnc_no: int, gave: int ... 3 more fields]
performer: DataFrame = [perf_no: int, perf_is: int ... 2 more fields]
place: DataFrame = [place_no: int, place_town: string ... 1 more field]
plays_in: DataFrame = [player: int, band_id: int]

## 11.
**List the name and town of birth of any performer born in the same city as James First.**

In [5]:
(musician.filter(musician("m_name") !== "James First")
 .join(musician.filter(musician("m_name") === "James First")
       .join(place, (musician("born_in") === place("place_no")))
       .select("born_in", "place_town"),
       "born_in")
 .select("m_name", "place_town")
 .orderBy("m_name")
 .showHTML())

m_name,place_town
Alan Fluff,London
Andy Jones,London
Theo Mengel,London


## 12.
**Create a list showing for EVERY musician born in Britain the number of compositions and the number of instruments played.**

In [6]:
val t = (musician
     .join(place.filter(place("place_country").isin("England", "Scotland")),
           (musician("born_in") === place("place_no")), joinType="left")
     .join(composer, (musician("m_no") === composer("comp_is")), 
           joinType="left")
     .join(has_composed, (composer("comp_no") === has_composed("cmpr_no")), 
           joinType="left")
     .join(composition, (has_composed("cmpn_no") === composition("c_no")), 
           joinType="left")
     .groupBy("m_no", "m_name")
     .agg(count("c_no").alias("n_composition")))
val p = (musician.join(performer, (musician("m_no") === performer("perf_is")),
                       joinType="left")
     .select("m_no", "instrument")
     .distinct()
     .groupBy("m_no")
     .agg(count("instrument").alias("n_instrument")))
t.join(p, "m_no").orderBy("m_name").showHTML()

m_no,m_name,n_composition,n_instrument
14,Alan Fluff,0,2
19,Andy Jones,1,0
12,Davis Heavan,0,3
18,Elsie James,0,3
1,Fred Bloggs,2,0
17,Freda Miles,2,0
4,Harriet Smithson,0,2
8,Harry Forte,2,3
3,Helen Smyth,1,1
5,James First,4,1


t: DataFrame = [m_no: int, m_name: string ... 1 more field]
p: DataFrame = [m_no: int, n_instrument: bigint]

## 13.
**Give the band name, conductor and contact of the bands performing at the most recent concert in the Royal Albert Hall.**

In [7]:
val rah = (concert.filter(concert("concert_venue") === "Royal Albert Hall")
           .orderBy(col("con_date").desc)
           .limit(1))
(band
 .join(musician.withColumnRenamed("m_name", "contact")
       .alias("mus1"), 
       (band("band_contact") === col("mus1.m_no")))
 .join(performance, (band("band_no") === performance("gave")))
 .join(musician.withColumnRenamed("m_name", "conductor")
       .alias("mus2"),
       (performance("conducted_by") === col("mus2.m_no")))
 .join(rah, (performance("performed_in") === rah("concert_no")))
 .select("band_name", "contact", "conductor")
 .showHTML())

band_name,contact,conductor
Somebody Loves this,Theo Mengel,Alan Fluff


rah: Dataset[Row] = [concert_no: int, concert_venue: string ... 3 more fields]

## 14.
**Give a list of musicians associated with Glasgow. Include the name of the musician and the nature of the association - one or more of "LIVES_IN", "BORN_IN", "PERFORMED_IN" AND "IN_BAND_IN".**

In [8]:
val t1 = (musician.join(place, (musician("born_in") === place("place_no")))
      .select("m_name", "place_town")
      .withColumn("assoc", lit("BORN_IN")))
val t2 = (musician.join(place, (musician("living_in") === place("place_no")))
      .select("m_name", "place_town")
      .withColumn("assoc", lit("LIVES_IN")))
val t3 = (musician.join(performer, (musician("m_no") === "perf_is"))
      .join(plays_in, (performer("perf_no") === plays_in("player")))
      .join(performance, (plays_in("band_id") === performance("gave")))
      .join(concert, (performance("performed_in") === concert("concert_no")))
      .join(place, (concert("concert_in") === place("place_no")))
      .select("m_name", "place_town")
      .withColumn("assoc", lit("PERFORMED_IN")))
val t4 = (musician.join(plays_in, (musician("m_no") === plays_in("player")))
      .join(band, (plays_in("band_id") === band("band_no")))
      .join(place, (band("band_home") === place("place_no")))
      .select("m_name", "place_town")
      .withColumn("assoc", lit("IN_BAND_IN")))
(t1.union(t2).union(t3).union(t4)
 .filter(col("place_town") === "Glasgow")
 .select("m_name", "assoc")
 .distinct()
 .showHTML())

m_name,assoc
Louise Simpson,BORN_IN
Steven Chaytors,BORN_IN
Lovely Time,BORN_IN
Jeff Dawn,LIVES_IN
Harriet Smithson,LIVES_IN
James Steeple,LIVES_IN
Andy Jones,LIVES_IN
Louise Simpson,LIVES_IN
Lovely Time,IN_BAND_IN
Jeff Dawn,IN_BAND_IN


t1: DataFrame = [m_name: string, place_town: string ... 1 more field]
t2: DataFrame = [m_name: string, place_town: string ... 1 more field]
t3: DataFrame = [m_name: string, place_town: string ... 1 more field]
t4: DataFrame = [m_name: string, place_town: string ... 1 more field]

## 15.
**Jeff Dawn plays in a band with someone who plays in a band with Sue Little. Who is it and what are the bands?**

In [9]:
val sue_band = (musician.filter(musician("m_name") === "Sue Little")
            .join(performer, (musician("m_no") === performer("perf_is")))
            .join(plays_in, (performer("perf_no") === plays_in("player")))
            .join(band, (plays_in("band_id") === band("band_no")))
            .select("band_id", "band_name"))
val sue_coplayers = (musician
                 .join(performer, (musician("m_no") === performer("perf_is")))
                 .join(plays_in, (performer("perf_no") === plays_in("player")))
                 .join(sue_band, "band_id"))
val jeff_band = (musician.filter(musician("m_name") === "Jeff Dawn")
             .join(performer, (musician("m_no") === performer("perf_is")))
             .join(plays_in, (performer("perf_no") === plays_in("player")))
             .join(band, (plays_in("band_id") === band("band_no")))
             .select("band_id", "band_name"))
val jeff_coplayers = (musician
                 .join(performer, (musician("m_no") === performer("perf_is")))
                 .join(plays_in, (performer("perf_no") === plays_in("player")))
                 .join(jeff_band, "band_id"))
(sue_coplayers.select(
    col("m_no"), col("band_name").alias("Sue\"s band"), col("m_name"))
 .join(jeff_coplayers
       .select(col("m_no"), col("band_name").alias("Jeff\"s band")), "m_no")
 .select("m_name", "Sue\"s band", "Jeff\"s band")
 .showHTML())

m_name,"Sue""s band","Jeff""s band"
John Smith,BBSO,AASO


sue_band: DataFrame = [band_id: int, band_name: string]
sue_coplayers: DataFrame = [band_id: int, m_no: int ... 11 more fields]
jeff_band: DataFrame = [band_id: int, band_name: string]
jeff_coplayers: DataFrame = [band_id: int, m_no: int ... 11 more fields]

In [10]:
spark.stop()